This notebook needs to be finished and uploaded on the link https://nextcloud.univ-lille.fr/index.php/s/EWEZLFEEJNfaT8s by March 3rd, 2023. Please use Name_FirstName as the name of the notebook or folder. 

# Exploration-Exploitation in Reinforcement Learning

In this tutorial, we will implement the **UCBVI** algorithm, for exploration in MDPs with finite states and actions and a **finite horizon** criterion. In a finite horizon criterion, the value function of a policy is

$$V_h^{\pi}(s) = \mathbb{E}^{\pi}\left[\left.\sum_{\ell = h}^{H} \gamma^{\ell-h} r_{\ell} \right| s_h = s\right]$$

where the discount parameter $\gamma \in (0,1]$ is often set to $\gamma = 1$. 

### Useful libraries

In [ ]:
# initialize display and import function to show videos
from rlberry.colab_utils.display_setup import show_video

In [ ]:
# other useful imports
import numpy as np
import numba
import matplotlib.pyplot as plt
from copy import deepcopy
import gym
from rlberry.wrappers import DiscretizeStateWrapper
from gym.wrappers import Monitor

In [ ]:
gym.__version__

In [ ]:
%%javascript
(function(on) {
const e=$( "<a>Setup failed</a>" );
const ns="js_jupyter_suppress_warnings";
var cssrules=$("#"+ns);
if(!cssrules.length) cssrules = $("<style id='"+ns+"' type='text/css'>div.output_stderr { } </style>").appendTo("head");
e.click(function() {
    var s='Showing';  
    cssrules.empty()
    if(on) {
        s='Hiding';
        cssrules.append("div.output_stderr, div[data-mime-type*='.stderr'] { display:none; }");
    }
    e.text(s+' warnings (click to toggle)');
    on=!on;
}).click();
$(element).append(e);
})(true);

## Environment

Our goal is to learn a good policy in a Mountain Car environment. The Mountain Car environement as implemented in gym has a continuous state space. In order to apply UCBVI, we will discretize it (using a <a href="https://github.com/rlberry-py/rlberry/blob/main/rlberry/wrappers/discretize_state.py">wrapper from the rlberry library</a>).

In [ ]:
def render(env, horizon=180,policy=None):
  """
  input  
  horizon : length of the simulation 
  policy : either a determinstic policy represented by an (H,S) array 
  or a random policy which is uniform (None)
  """
  env = deepcopy(env)
  env = Monitor(env, './gym_videos', force=True, video_callable=lambda episode: True)
  for episode in range(1):
    done = False
    state = env.reset()
    env.render()
    for hh in range(horizon):
        if policy is not None:
          action = policy[hh, state]
        else:
          action = env.action_space.sample()
        state, reward, done, info = env.step(action)
        env.render()
    env.close()
    #show_video(directory='gym_videos')

In [ ]:
class MountainCatRewardWrapper(gym.Wrapper):
    def __init__(self, env):
        gym.Wrapper.__init__(self, env)

    def step(self, action):
        next_state, reward, done, info = self.env.step(action)
        if done:
            reward = 1.0
        else:
            reward = 0.0
        done = False 
        return next_state, reward, done, info

def get_mountain_car_env():
  env_with_continuous_states = MountainCatRewardWrapper(gym.make('MountainCar-v0'))
  env = DiscretizeStateWrapper(env_with_continuous_states, n_bins=10)
  return env

env = get_mountain_car_env()
render(env) #saves the video

In [ ]:
X = env.discretized_states[0,:] # discretized positions
Xdot = env.discretized_states[1,:] # discretized velocities

test = 67

print(env.observation_space)
print(env.action_space)
print("state ",test," is ", (X[test],Xdot[test])) 
show_video(directory='gym_videos') # play the video

# Implementation of backward induction (i.e. value iteration)

In a finite-horizon MDP, the optimal Bellman equations given a recursion that can be used to compute the optimal value function. We have $V_{H+1}^\star(s) = 0$ for all $s$ and for $h \leq H$, 

$$Q^\star_h(s,a) = r(s,a) + \gamma \sum_{s' \in \mathcal{S}} p(s'|s,a) V^\star_{h+1}(s') \ \ \text{and } \ \ V^\star_{h}(s) = \max_{a \in \mathcal{A}} Q_h^\star(s,a).$$

Recall that the optimal policy is deterministic but *non-stationary* and satisfies $\pi^\star_h(s) = \text{argmax}_{a} Q^\star_h(s,a)$. 

**Complete the code below in order to compute the optimal Q function in a finite-horizon MDP.**

Note that this code will also be useful to compute the policy used in each episode by UCB-VI, where we have to perform Value Iteration in an optimistic MDP.

In [ ]:
@numba.jit(nopython=True)  # use this to make the code much faster!

def backward_induction(P, R, H, gamma=1.0):
    """
    Parameters:
        P: transition function (S,A,S)-dim matrix
        R: reward function (S,A)-dim matrix
        H: horizon
        gamma: discount factor. Usually set to 1.0 in finite-horizon problems

    Returns:
        The optimal Q-function: array of shape (horizon, S,A)      
    """
    S, A = P.shape[0], P.shape[1]
    V = np.zeros((H + 1, S))
    Q = np.zeros((H+1,S,A))
    for h in range(H-1, -1, -1):
        for s in range(S):
            ## TODO: COMPUTE (Q[h,s,a])_{a} and V[h,s]
            # ... and clip the value (needed later in UCB-VI)
            if (V[h, s] > H - h):
                V[h, s] = H - h
    return Q

We cannot try this function on the moutain car environement, as the expected rewards and transition probabilities cannot be easily computed, and are not embedded in the environment, unlike in our previous GridWorld example. 

So you can check your code on a simple gridworld. 

In [ ]:
# Testing the implementation in a GridWorld
from rlberry.envs import GridWorld

test_env = GridWorld(nrows=8, ncols=8)
H = 50 # pick an horizon which is sufficient to reach the goal

Q_test = backward_induction(test_env.P,test_env.R,H,gamma=1.0)

state = test_env.reset()
test_env.enable_rendering()
for h in range(H):   
  action = np.argmax(Q_test[h, state,:])
  next_state, reward, is_terminal, info = test_env.step(action)
  if is_terminal:
    break
  state = next_state

# save video (run next cell to visualize it)
test_env.save_video('./videos/value_iteration_gw.mp4', framerate=10)
# clear rendering data
test_env.clear_render_buffer()
test_env.disable_rendering()


In [ ]:
# see video
show_video(filename='./videos/value_iteration_gw.mp4')

# Implementation of UCBVI

The UCBVI algorithm works as follows:

* In each episode $t$, the agent has observed $n_t$ transitions $(s_i, a_i, r_i, s_{i+1})_{i=1}^{n_t}$ of states, actions, rewards and next states.
* We estimate a model of the MDP as:
$$
\mathbf{rewards:}\quad\widehat{R}_t(s, a) = \frac{1}{N_t(s, a)} \sum_{i=1}^{n_t} \mathbb{1}\{s = s_i, a = a_i)\} r_i
\\
\mathbf{transitions:}\quad \widehat{P}_t(s'|s, a) =  \frac{1}{N_t(s, a)} \sum_{i=1}^{n_t} \mathbb{1}\{s = s_i, a = a_i, s'=s_{i+1}\} 
$$
where
$$
N_t(s, a) = \max\left(1, \sum_{i=1}^{n_t} \mathbb{1}\{s = s_i, a = a_i)\} \right)
$$
* We define exploration bonuses as
$$
B_t(s, a) \propto \sqrt{\frac{1}{N_t(s, a)}} \cdot
$$

* Then, in episode $t$, we compute $\{Q_h^t(s, a)\}_{h=1}^H$ as the ($H$-horizon) optimal value functions in the MDP whose transitions are $\widehat{P}_t$ and whose rewards are $(\widehat{R}_t + B_t)$. At step $h$ of episode $t$, the agent chooses the action $a_h^t \in \arg\max_a Q_h^t(s, a)$.

In [ ]:
# An example of bonus function
def bonus(N): 
    """input : a numpy array (nb of visits)
    output : a numpy array (bonuses)"""
    nn = np.maximum(N, 1)
    return np.sqrt(1.0/nn)

# The UCB-VI algorithm
def UCBVI(env,H, nb_episodes,bonus_function=bonus,gamma=1):
    """
    Parameters:
        env: environement
        bonus_function : maps the number of visits to the corresponding bonus
        H: horizon
        gamma: discount factor. Usually set to 1.0 in finite-horizon problems

    Returns:
        episode_rewards: a vector storing the sum of rewards obtained in each episode 
        N_sa : array of size (S,A) giving the total number of visits in each state
        Rhat : array of size (S,A) giving the estimated average rewards
        Phat : array of size (S,A,S) giving the estimated transition probabilities
        optimistic_Q : array of size (H,S,A) giving the optimistic Q function used in the last episode     
    """
    S = env.observation_space.n
    A = env.action_space.n
    ## TO BE COMPLETED
    
    return episode_rewards, N_sa, Rhat, Phat,optimistic_Q

To check whether the algorithm is working, your can can UCB-VI for a large number of episodes and monitor the amount of reward collected during the episode and the total number of visited states every 50 episodes. The algorithm should be improving, both in terms of maximizing rewards and in terms of exploration. 

In [ ]:
NUM_REPETITIONS = 1
HORIZON = 180
NUM_EPISODES = 500

env = get_mountain_car_env()

rewards = np.zeros((NUM_REPETITIONS, NUM_EPISODES))
for sim in range(NUM_REPETITIONS):
    print(f"Running simulation: {sim}")
    rewards[sim], N_sa, Rhat, Phat, optimistic_Q = UCBVI(env, H=HORIZON, nb_episodes=NUM_EPISODES)

# Evaluation of UCB-VI 

The theoretical guarantees for UCB-VI are in terms of regret, so the algorithm is aimed at collecting a large amount of rewards during learning. However, it can still provide a guess for the optimal policy and an estimate of its value.  

Two particular policies may be of interest: 
- the optimistic policy, which is the greedy policy wrt the optimistic Q-Value in the last episode 
- the empirical optimal policy, which is the optimal policy in the empirical MDP given by Rhat and P_hat

**Which one seems more interesting to you? For each policy, visualize how it behaves in the environment and display an estimate of the optimal value function. Comment on its shape.**

In [ ]:
policy = np.argmax(optimistic_Q,axis=2)
render(env, HORIZON,policy)
show_video(directory='gym_videos')

value = np.max(optimistic_Q,axis=2)
plt.scatter(env.discretized_states[0, :], env.discretized_states[1, :], c=value[0, :], s=400)
plt.xlabel('Car Position')
plt.ylabel('Car Velocity')
clb=plt.colorbar()
clb.ax.set_title('Value')
plt.show()

UCB-VI is supposed to be good at maximizing rewards. Now we can display the total reward obtained in $k$ episodes as a function of the episode $k$ for different variants of UCB-VI, using different bonusses (e.g. a bonus closer to a theoretically-valid one, or no bonus at all) or a stupid uniform baseline. 

Note that unlike what we did for bandit algorithm, we cannot compute regret as we do not have access to the optimal value function for this problem. However, it may still be good to display average rewards over multiple simulations. 

**Display a cumulative rewards curve. What is a good bonus for this problem?**

# Going further (theory)

*On discount versus finite horizon.* In the previous practical sessions, we were finding an optimal policy wrt to a discounted notion of value. For a task like Mountain Car, all we care about is learning the policy that climbs up hill. We showed that choosing $H$ large enough in a finite horizon MDP does it. 

**What discount would you have chosen for a discounted-based algorithm?**  



*On the use of exploration bonus beyond UCB-VI.* Replacing rewards by rewards + a bonus that decays with the number of visits is a generic idea that can be transposed to any online reinforcement learning algorithm. 

**Would this idea make sense for the Fitted-Q algorithm? In what other algorithm(s) could you have done it?**

# Going further (practice)

Implement another RL algorithm of your choice for this problem. Compare it to UCBVI.  